In [8]:
import pandas as pd
import numpy as np
import sklearn

In [9]:
class Node():
    def __init__(self, feature=None, split_point=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node. Contains variables for all the specific node
        self.feature = feature          # Features in the dataset
        self.split_point = split_point  # The value of a feature to split on
        self.left = left
        self.right = right
        self.info_gain = info_gain      # Calculate information gain for entropy and also used this variable to get the lowest gini impurity   
        
        # for leaf node
        self.value = value              # The majority class of a leaf node

In [18]:
class DecisionTreeClassifier():
    def __init__(self, mode_to_use, min_samples_split=1):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        # gini impurity or entropy to select the best feature to split
        self.mode_to_use = mode_to_use

    
    def build_tree(self, dataset):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if the information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"])
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"])
                # return decision node
                return Node(best_split["feature_index"], best_split["split_point"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_split_points = np.unique(feature_values)
            # loop over all the feature values present in the data
            
            for split_point in possible_split_points:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, split_point)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, self.mode_to_use)
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["split_point"] = split_point
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, split_point):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=split_point])
        dataset_right = np.array([row for row in dataset if row[feature_index]>split_point])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode_to_use):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)    # left branch of a split
        weight_r = len(r_child) / len(parent)    # rightt branch of a split
        if mode_to_use=="gini":
            # The best feature to split on will be with the lowest gini impurity. So I am subtracting the gini index from positive infinity to
            # find out what feature has the highest gain which is equal to lowest impurity
            gain = float('inf') - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child)) 
        if mode_to_use=="entropy":
            # The best feature to split on is the one with the highest information gain for entropy
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)            # Get how many unique classes there are
        entropy = 0                            
        for cls in class_labels:               # Iterate over all the classes
            p_cls = len(y[y == cls]) / len(y)  # The number of instances of a class divided by the total of all the class / probability of a class 
            entropy += -p_cls * np.log2(p_cls) # entropy calculation
        return entropy

        
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)             # Get how many unique classes there are
        gini = 0
        for cls in class_labels:                # Iterate over all the classes
            p_cls = len(y[y == cls]) / len(y)   # The number of instances of a class divided by the total of all the class / probability of a class
            gini += p_cls**2 
        return 1 - gini                         # gini index
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)                   
        return max(Y, key=Y.count)               # Number of majority class in a node.
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)  
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature]
        if feature_val<=tree.split_point:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

### Problem 2 : Building models for the given dataset

In [19]:
df = pd.read_csv("C:\\Users\\razic\\OneDrive\\Desktop\\IDAI.610.01 - Fundamentals of AI\\problem set 1\\ps1-610\\ps1\\Final_data\\wdbc_dev_normalized.csv")
X_train = df.iloc[:,:-1].values
Y_train = df.iloc[:,-1:].values

In [20]:
test_df = pd.read_csv("C:\\Users\\razic\\OneDrive\\Desktop\\IDAI.610.01 - Fundamentals of AI\\problem set 1\\ps1-610\\ps1\\Final_data\\wdbc_test_normalized.csv")
X_test = test_df.iloc[:,:-1].values
Y_test = test_df.iloc[:,-1:].values

### Fit and calculate accuracy for gini

In [21]:
classifier_gini = DecisionTreeClassifier(min_samples_split=2, mode_to_use= 'gini')
classifier_gini.fit(X_train,Y_train)

In [22]:
Y_pred_gini = classifier_gini.predict(X_test) 
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test, Y_pred_gini))

0.7894736842105263


### Fit and calculate accuracy for entropy

In [23]:
classifier_entropy = DecisionTreeClassifier(min_samples_split=2, mode_to_use= 'entropy')
classifier_entropy.fit(X_train,Y_train)

In [24]:
Y_pred_entropy = classifier_entropy.predict(X_test) 
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test, Y_pred_entropy))

0.8771929824561403


### Problem 3 : Compare with an implementation in an ML library

#### The below code is copied from the given notebook

In [33]:
import graphviz
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

In [34]:
np.random.seed(seed=42)

In [35]:
# First run with wdbc_*_normalized.csv and also compare the result with wdbc_*_raw.csv data
# Loading training, tuning, and test data
df_train = pd.read_csv("C:\\Users\\razic\\OneDrive\\Desktop\\IDAI.610.01 - Fundamentals of AI\\problem set 1\\ps1-610\\ps1\\Final_data\\wdbc_train_normalized.csv")
df_dev = pd.read_csv("C:\\Users\\razic\\OneDrive\\Desktop\\IDAI.610.01 - Fundamentals of AI\\problem set 1\\ps1-610\\ps1\\Final_data\\wdbc_dev_normalized.csv")
df_test = pd.read_csv("C:\\Users\\razic\\OneDrive\\Desktop\\IDAI.610.01 - Fundamentals of AI\\problem set 1\\ps1-610\\ps1\\Final_data\\wdbc_test_normalized.csv")

In [36]:
df_train

,Radius,Texture,Perimeter,Area,Smoothness,Compactness,Concavity,ConcavePoints,Symmetry,FractalDimension,...,worstTexture,worstPerimeter,worstArea,worstSmoothness,worstCompactness,worstConcavity,worstConcavePoints,worstSymmetry,worstFractalDimension,Diagnosis
0,1.828212,-0.353322,1.684473,1.907030,-0.826235,-0.486643,-0.023825,0.547662,0.001391,-0.867889,...,-0.368879,1.533776,1.888827,-0.375282,-0.430066,-0.146620,1.086129,-0.243675,0.280943,M
1,1.578499,0.455786,1.565126,1.557513,0.941382,1.052000,1.362280,2.035440,0.938859,-0.397658,...,-0.023953,1.346291,1.455004,0.526944,1.081980,0.854222,1.953282,1.151242,0.201214,M
2,-0.768233,0.253509,-0.592166,-0.763792,3.280667,3.399917,1.914213,1.450431,2.864862,4.906602,...,0.133866,-0.249720,-0.549538,3.391291,3.889975,1.987839,2.173873,6.040726,4.930672,M
3,1.169878,0.160508,1.137124,1.094332,-0.123028,0.088218,0.299809,0.646366,-0.064268,-0.761662,...,0.322599,1.367122,1.274098,0.518184,0.021196,0.509104,1.195664,0.262245,-0.014718,M
4,-0.319885,0.588312,-0.183919,-0.383870,2.199903,1.682529,1.218025,1.149680,1.963872,1.571079,...,0.822090,-0.031581,-0.248145,1.661295,1.816711,1.278909,1.390393,2.387756,1.287517,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,-0.830662,2.343703,-0.876540,-0.764076,-1.555040,-1.301977,-1.113893,-1.260710,-2.741705,-1.101588,...,2.053734,-0.954428,-0.774528,-1.738693,-1.266871,-1.304683,-1.743529,-2.157444,-1.378409,B
337,2.109139,0.720838,2.058974,2.341795,1.040926,0.218868,1.945573,2.318924,-0.312314,-0.930209,...,0.117596,1.751022,2.013529,0.378033,-0.273077,0.663928,1.627719,-1.358963,-0.708467,M
338,1.703356,2.083301,1.614511,1.722326,0.102368,-0.017817,0.692434,1.262558,-0.217473,-1.057681,...,2.045599,1.420690,1.493644,-0.690623,-0.394473,0.236365,0.733182,-0.531387,-0.973122,M
339,0.701667,2.043775,0.672084,0.577445,-0.839745,-0.038646,0.046547,0.105684,-0.808406,-0.894800,...,1.373645,0.578492,0.427529,-0.808876,0.350427,0.326479,0.413705,-1.103578,-0.318129,M


In [37]:
df_train.columns[:-1].tolist()

['Radius',
 'Texture',
 'Perimeter',
 'Area',
 'Smoothness',
 'Compactness',
 'Concavity',
 'ConcavePoints',
 'Symmetry',
 'FractalDimension',
 'seRadius',
 'seTexture',
 'sePerimeter',
 'seArea',
 'seSmoothness',
 'seCompactness',
 'seConcavity',
 'seConcavePoints',
 'seSymmetry',
 'seFractalDimension',
 'worstRadius',
 'worstTexture',
 'worstPerimeter',
 'worstArea',
 'worstSmoothness',
 'worstCompactness',
 'worstConcavity',
 'worstConcavePoints',
 'worstSymmetry',
 'worstFractalDimension']

In [38]:
features = df_train.columns[:-1].tolist()

# Concatenation for PredefinedSplit below
x_train = np.concatenate([df_train[features], df_dev[features]])
y_train = np.concatenate([df_train["Diagnosis"], df_dev["Diagnosis"]])

x_test = df_test[features].to_numpy()
y_test = df_test["Diagnosis"].to_numpy()

In [39]:
tree_params = {'criterion':['gini','entropy'],'max_depth':list(np.arange(3,51))}

# The function PredefinedSplit ensures that the same dev set is used for development testing in the tuning phase
test_fold = [-1 for _ in range(df_train.shape[0])] + [0 for _ in range(df_dev.shape[0])]
ps = PredefinedSplit(test_fold)

clf = GridSearchCV(tree.DecisionTreeClassifier(), tree_params, cv=ps)
clf.fit(x_train, y_train)

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                       15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
                                       25, 26, 27, 28, 29, 30, 31, 32, ...]})

In [40]:
clf.best_params_

{'criterion': 'entropy', 'max_depth': 24}

In [41]:
clf.best_params_["criterion"]

'entropy'

In [42]:
# Use the best hyperparameters from the grid search to create the decision tree, using the training set
best_tree_model = tree.DecisionTreeClassifier(criterion=clf.best_params_['criterion'], max_depth=clf.best_params_['max_depth'])
best_tree_model.fit(df_train[features].to_numpy(), df_train["Diagnosis"].to_numpy())

DecisionTreeClassifier(criterion='entropy', max_depth=24)

In [43]:
y_pred = best_tree_model.predict(x_test)

print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           B       0.96      0.97      0.97        72
           M       0.95      0.93      0.94        42

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



#### Due to some installation issues graphviz doesn't work on my system

In [ ]:
#Use graphviz to print the tree with nice formatting
dot_data = tree.export_graphviz(best_tree_model, out_file=None, feature_names=features, rounded=True,class_names=["B","M"],precision=2) 
graph = graphviz.Source(dot_data)
graph

In [46]:
feature_importances = best_tree_model.feature_importances_
importance_dict = {feature: importance for feature, importance in zip(features, feature_importances)}
df_feature_imp = pd.DataFrame(list(importance_dict.items()), columns=['Feature', 'Gini Importance'])

df_feature_imp.sort_values('Gini Importance', ascending=False)

,Feature,Gini Importance
22,worstPerimeter,0.611368
27,worstConcavePoints,0.195786
21,worstTexture,0.064628
13,seArea,0.037489
19,seFractalDimension,0.026465
14,seSmoothness,0.013881
4,Smoothness,0.012751
1,Texture,0.012007
11,seTexture,0.009991
20,worstRadius,0.008482
